In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GITHUB_TOKEN"] = "YOUR_GITHUB_TOKEN"
Github_repo_name =  os.getenv("GITHUB_REPO")


In [5]:
import google.generativeai as genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-1.5-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [ ]:
model.

AttributeError: 'GenerativeModel' object has no attribute 'invoke'

In [7]:
def diagnosis_agent(error_text: str):
    prompt = f"""
    You are a CI/CD diagnosis agent. 
    Input error logs:
    {error_text}

    Analyze the logs and return JSON:
    {{
        "root_cause": "...",
        "problem_file": "...",
        "fix_description": "..."
    }}
    """
    resp = model.generate_content(prompt)
    return resp.text


In [8]:
def fixing_agent(diagnosis_json: str):
    prompt = f"""
    You are an AI code-fixing agent.

    Based on this diagnosis:
    {diagnosis_json}

    Provide:
    {{
      "file_to_modify": "...",
      "new_branch_name": "...",
      "patch_diff": "..."
    }}

    - patch_diff must be a unified diff.
    """
    resp = model.generate_content(prompt)
    return resp.text


In [ ]:
from github import Github
import base64

g = Github(os.environ["GITHUB_TOKEN"])
repo = g.get_user().get_repo(Github_repo_name)
